# LittleMCMC Quickstart

LittleMCMC is a lightweight and performant implementation of HMC and NUTS in Python, spun out of the PyMC project. In this quickstart tutorial, we will introduce LittleMCMC

## Who should/should not use LittleMCMC?

<div class="alert alert-block alert-info">
LittleMCMC is a fairly bare bones library with a very niche use case. Most users will probably find that [PyMC3](https://github.com/pymc-devs/pymc3) will satisfy their needs, with better strength of support and quality of documentation.
</div>

If you:

1. Have model with only continuous parameters,
1. Are willing to manually "unconstrain" all of your model's parameters (if necessary),
1. Have methods to compute the log probability of the model and its derivative, exposed via a Python callable,
1. And all you need is an implementation of HMC/NUTS (preferably in Python) to sample from your model

then you should consider using LittleMCMC!

## Sampling

In [1]:
import numpy as np
import scipy
import littlemcmc as lmc

In [2]:
def logp_func(x, loc=0, scale=1):
    return np.log(scipy.stats.norm.pdf(x, loc=loc, scale=scale))


def dlogp_func(x, loc=0, scale=1):
    return -(x - loc) / scale


def logp_dlogp_func(x, loc=0, scale=1):
    return logp_func(x, loc=loc, scale=scale), dlogp_func(x, loc=loc, scale=scale)

In [3]:
trace, stats, results = lmc.sample(
    logp_dlogp_func=logp_dlogp_func,
    size=1,
    draws=1000,
    tune=500,
    step=lmc.NUTS(logp_dlogp_func=logp_dlogp_func, size=1),
    chains=4,
    cores=4,
    progressbar="notebook"
)

/Users/george/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


### Inspecting the Output of `lmc.sample`

In [4]:
trace

array([-0.30283875,  1.00489538,  1.00489538, ...,  0.25364023,
        1.42310791,  1.42310791])

In [5]:
trace.shape

(4000,)

In [13]:
stats = [iteration_stats for (_, chain_stats) in results for iteration_stats in chain_stats]

In [23]:
foo = {
    "tune": np.array([iteration_stats["tune"] for iteration_stats in stats]).astype(int),
    "diverging": np.array([iteration_stats["diverging"] for iteration_stats in stats]).astype(int),
    "depth": np.array([iteration_stats["depth"] for iteration_stats in stats]).astype(int),
    "mean_tree_accept": np.hstack([iteration_stats["mean_tree_accept"] for iteration_stats in stats]),    
    "energy_error": np.hstack([iteration_stats["energy_error"] for iteration_stats in stats]),
    "energy": np.hstack([iteration_stats["energy"] for iteration_stats in stats]),
    "tree_size": np.array([iteration_stats["tree_size"] for iteration_stats in stats]).astype(int),
    "max_energy_error": np.hstack([iteration_stats["max_energy_error"] for iteration_stats in stats]),
    "model_logp": np.hstack([iteration_stats["model_logp"] for iteration_stats in stats]),
    "step_size": np.hstack([iteration_stats["step_size"] for iteration_stats in stats]),
    "step_size_bar": np.hstack([iteration_stats["step_size_bar"] for iteration_stats in stats]),
}

## Other Modules

These modules can be customized.